<a href="https://colab.research.google.com/github/Sesandi2003/parallel-programming/blob/main/matrix_multiplication_with_output_data_decomp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%%writefile matrix_mult.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

// Define matrix dimensions
#define N 1000 // Rows of A
#define M 1000 // Cols of A / Rows of B
#define P 1000 // Cols of B

int main() {
    // 1. Allocate memory for matrices as 1D arrays
    // size_t is used to prevent integer overflow for large matrices
    double *A = (double *)malloc(N * M * sizeof(double));
    double *B = (double *)malloc(M * P * sizeof(double));
    double *C = (double *)malloc(N * P * sizeof(double));

    if (A == NULL || B == NULL || C == NULL) {
        printf("Memory allocation failed.\n");
        return 1;
    }

    // 2. Initialize matrices
    // A and B get values, C gets 0
    for (int i = 0; i < N * M; i++) A[i] = 1.0;
    for (int i = 0; i < M * P; i++) B[i] = 2.0;
    for (int i = 0; i < N * P; i++) C[i] = 0.0;

    printf("Starting Matrix Multiplication (%dx%d by %dx%d)...\n", N, M, M, P);
    printf("Max Threads available: %d\n", omp_get_max_threads());

    // Start Timer (OpenMP wall clock)
    double start_time = omp_get_wtime();

    // 3. Parallel Matrix Multiplication
    // We parallelize the outer loop (rows of A / rows of C)
    #pragma omp parallel for schedule(static)
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < P; j++) {
            double sum = 0.0;
            for (int k = 0; k < M; k++) {
                // Access using Row-Major Indexing: A[row * num_cols + col]
                sum += A[i * M + k] * B[k * P + j];
            }
            C[i * P + j] = sum;
        }
    }

    // Stop Timer
    double end_time = omp_get_wtime();
    printf("Computation finished in: %f seconds.\n", end_time - start_time);

    // 4. Verification
    // Expected value = M * 1.0 * 2.0 = 2000.0
    printf("C[0][0] = %.2f (Expected: %.2f)\n", C[0], (double)M * 2.0);

    // 5. Clean up
    free(A);
    free(B);
    free(C);

    return 0;
}

Writing matrix_mult.c


In [12]:
!gcc -fopenmp matrix_mult.c -o matrix_mult
!./matrix_mult

Starting Matrix Multiplication (1000x1000 by 1000x1000)...
Max Threads available: 2
Computation finished in: 7.668556 seconds.
C[0][0] = 2000.00 (Expected: 2000.00)
